<a href="https://colab.research.google.com/github/DhruvaBansal00/ConfidentMT/blob/master/driverTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/My Drive/ConfidentMachineTranslation/flores
!pwd
!ls
import sys
sys.path.append('/content/drive/My Drive/ConfidentMachineTranslation/flores/src/')
!pip install fairseq sacrebleu sentencepiece fastBPE sacremoses xgboost

Mounted at /content/drive
/content/drive/My Drive/ConfidentMachineTranslation/flores
/content/drive/My Drive/ConfidentMachineTranslation/flores
analysis	       DeprecatedColabScripts	 prepare-sien.sh
backward_models        driver.ipynb		 Resources
checkpoints	       driverTransformers.ipynb  scripts
ClassificationDataset  Ensembles		 sentencepiece_models
configs		       language_models		 src
data		       LoadEn-Si.ipynb
data-bin	       noisychannel


In [2]:
# prints how much GPU RAM is available
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.7 GB  | Proc size: 159.5 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [2]:
##Methods for CLI with fairseq - WIP
import os
import glob
from tqdm.notebook import tqdm

class FairseqWrapper:

    def runFairseqGenerate(self, dataFolder, sourceLang, targetLang, fwModel, beam, lenpen, dataSet, removeBpe, saveFile):
        !fairseq-generate $dataFolder --source-lang $sourceLang --target-lang $targetLang \
        --path $fwModel --beam $beam --lenpen $lenpen --gen-subset $dataSet --remove-bpe=$removeBpe --sacrebleu > $saveFile

    def runFairseqScore(self, translations, groundTruth, saveFile, metric):
        !fairseq-score --sys $translations --ref $groundTruth --$metric > $saveFile
    
    def runFairseqPreprocessLM(self, srcdict, preprocCommand, bpeTranslations, dest):
        !fairseq-preprocess --only-source --srcdict $srcdict --$preprocCommand $bpeTranslations \
        --destdir $dest --workers 20
    
    def runFairseqPreprocessBinarize(self, srcLang, trgLang, srcdict, preprocCommand, bpeTranslations, destdir):
        !fairseq-preprocess --source-lang $srcLang --target-lang $trgLang --srcdict $srcdict\
        --$preprocCommand $bpeTranslations --destdir $destdir --joined-dictionary --workers 20

    def runFairseqEvalLM(self, preprocFile, lmModel, batchSize, maxTokens, dataSet, saveFile):
        !fairseq-eval-lm $preprocFile --path $lmModel --batch-size $batchSize --max-tokens $maxTokens \
        --output-word-probs --sample-break-mode eos --gen-subset $dataSet > $saveFile

    def deleteFolder(self, folder):
        print("Deleting contents in folder: " + str(folder))
        for f in tqdm(glob.glob(folder)):
            !rm $f


In [3]:
import translationUtils
from translation import Translation
import dataUtils
import classification
import thresholding
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import random
import regression
import transformerUtils 

trainTranslations = dataUtils.createObjectsFromFile("valid")
testTranslations = dataUtils.createObjectsFromFile("test")

In [6]:
trainData, trainLabels, testData, testLabels = transformerUtils.createDataset(trainTranslations, testTranslations, 15)